### 0 使用CPU进行模型的测试与转换

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import keras
from keras.datasets import mnist
import numpy as np
import cv2

### 1 使用自定义图片测试.h5模型

In [2]:
# 加载模型
model=models.load_model('alexnet.h5')
# 读取测试数据-10张手写数字图片-预处理
num_image=list()
for i in range(10):
    img_path='handwrite_num/{}.png'.format(i)
    num_image.append(cv2.resize(cv2.imread(img_path,cv2.IMREAD_COLOR).astype('float32'),(227,227))/255.0)
input_image = np.array(num_image)
# 对模型进行预测
predictions = model.predict(input_image)
# 获取预测结果中概率最高的类别
for i in range(10):
    print("num",i,"预测结果：", np.argmax(predictions[i]))

1/1 [==============================] - 0s 390ms/step
num 0 预测结果： 0
num 1 预测结果： 1
num 2 预测结果： 2
num 3 预测结果： 3
num 4 预测结果： 4
num 5 预测结果： 5
num 6 预测结果： 6
num 7 预测结果： 7
num 8 预测结果： 8
num 9 预测结果： 9


### 2 将模型转换为tflite格式并保存

In [3]:
# 将keras模型转换为tflite格式
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
# 保存alexnet-tflite模型
tflite_name = "alexnet.tflite"
open(tflite_name, "wb").write(tflite_model)

INFO:tensorflow:Assets written to: D:\USER_T~1\tmp1tvk6etg\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmp1tvk6etg\assets


233308828

### 3 测试tflite格式的alexnet模型

In [4]:
# 加载TFLite模型
interpreter = tf.lite.Interpreter(model_path='alexnet.tflite')
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 读取测试数据-10张手写数字图片-预处理
num_image=list()
for i in range(10):
    img_path='handwrite_num/{}.png'.format(i)
    num_image.append(cv2.resize(cv2.imread(img_path,cv2.IMREAD_COLOR).astype('float32'),(227,227))/255.0)
input_data = np.array(num_image)

for i in range(10):
    test_image=input_data[i].reshape((1,227,227,3))
    # 设置输入张量的值
    interpreter.set_tensor(input_details[0]['index'], test_image)
    # 运行模型
    interpreter.invoke()
    # 获取输出张量的值
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print('num',i,'预测结果',np.argmax(output_data))


num 0 预测结果 0
num 1 预测结果 1
num 2 预测结果 2
num 3 预测结果 3
num 4 预测结果 4
num 5 预测结果 5
num 6 预测结果 6
num 7 预测结果 7
num 8 预测结果 8
num 9 预测结果 9


### 4 对h5格式的alexnet模型进行int8量化，并保存为tflite格式

In [5]:
# 加载Keras模型
keras_model = tf.keras.models.load_model('alexnet.h5')
# 将Keras模型转换为TfLite模型
converter_quant = tf.lite.TFLiteConverter.from_keras_model(keras_model)
# set the optimization parameters for TensorFlow Lite conversion
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]

# convert the model to TensorFlow Lite format with float32 activations and int8 weights
alexnet_quanitfied_model = converter_quant.convert()

#保存转换后的模型
alexnet_quanitfied_name = "alexnet_quanitfied.tflite"
open(alexnet_quanitfied_name, "wb").write(alexnet_quanitfied_model)

INFO:tensorflow:Assets written to: D:\USER_T~1\tmp8ezgmtq6\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmp8ezgmtq6\assets


58387408

### 5 测试int8量化后的AlexNet模型

In [8]:
# 加载量化后的TFLite模型
interpreter = tf.lite.Interpreter(model_path='alexnet_quanitfied.tflite')
interpreter.allocate_tensors()

# 获取输入和输出张量的索引
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 读取测试数据-10张手写数字图片-预处理
num_image=list()
for i in range(10):
    img_path='handwrite_num/{}.png'.format(i)
    num_image.append(cv2.resize(cv2.imread(img_path,cv2.IMREAD_COLOR).astype('float32'),(227,227))/255.0)
input_data = np.array(num_image)

for i in range(10):
    test_image=input_data[i].reshape((1,227,227,3))
    # 设置输入张量的值
    interpreter.set_tensor(input_details[0]['index'], test_image)
    # 运行模型
    interpreter.invoke()
    # 获取输出张量的值
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print('num',i,'预测结果',np.argmax(output_data))


num 0 预测结果 0
num 1 预测结果 1
num 2 预测结果 2
num 3 预测结果 3
num 4 预测结果 4
num 5 预测结果 5
num 6 预测结果 6
num 7 预测结果 7
num 8 预测结果 8
num 9 预测结果 9


### 6 错误

若采用以下方法进行int8量化，理论上可生成激活值（偏置）为int64、权重为int8的tflite模型

但是会遇到输入张量的维度不符合模型期望的问题，目前尚未解决。

In [9]:

# 加载Keras模型
keras_model = tf.keras.models.load_model('alexnet.h5')
# 将Keras模型转换为TfLite模型
converter_quant = tf.lite.TFLiteConverter.from_keras_model(keras_model)
# set the optimization parameters for TensorFlow Lite conversion
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT]
# 选择量化策略
converter_quant.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] 
# load the MNIST dataset and prepare it for representative data generation
mnist_train, _ = tf.keras.datasets.mnist.load_data("mnist.npz")
images = tf.cast(mnist_train[0], tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
# define a generator function for representative data
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]
# set the representative dataset for the converter
converter_quant.representative_dataset = representative_data_gen

# convert the model to TensorFlow Lite format with int64 activations and int8 weights
quantified_AlexNet_model = converter_quant.convert()

#保存转换后的模型
quantified_AlexNet_model_name = "quantified_AlexNet_model.tflite"
open(quantified_AlexNet_model_name, "wb").write(quantified_AlexNet_model)

INFO:tensorflow:Assets written to: D:\USER_T~1\tmpi7gwfnxd\assets


INFO:tensorflow:Assets written to: D:\USER_T~1\tmpi7gwfnxd\assets
d:\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\lite\python\convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


RuntimeError: tensorflow/lite/kernels/conv.cc:343 input->dims->size != 4 (3 != 4)Node number 0 (CONV_2D) failed to prepare.